In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to generate a list of dates from January 1, 2024, to October 16, 2024
def generate_date_range(start_date='2024-01-01', end_date='2024-10-16'):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    delta = timedelta(days=1)
    
    dates = []
    current_date = start
    while current_date <= end:
        dates.append(current_date)
        current_date += delta
    
    return dates

# Function to fetch historical weather data for a given date
def fetch_historical_weather(date):
    # Format the date as 'YYYY-MM-DD'
    date_str = date.strftime('%Y-%m-%d')
    
    # The historical API endpoint (replace with your API key)
    url = f"http://api.weatherapi.com/v1/history.json?key=eb49d67b54f44743b2f102519241710&q=Vijayawada&dt={date_str}"

    # Send a GET request to the API
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Extract necessary weather information from the dataset
        if 'forecast' in data and 'forecastday' in data['forecast']:
            forecast = data['forecast']['forecastday'][0]['day']
            
            precipitation = forecast['totalprecip_mm']
            temp_max = forecast['maxtemp_c']
            temp_min = forecast['mintemp_c']
            wind = forecast['maxwind_kph']
            condition = forecast['condition']['text'].lower()

            # Define weather condition classification (adjust based on needs)
            if "mist" in condition or "fog" in condition:
                weather_condition = "fog"
            elif "rain" in condition or "drizzle" in condition:
                weather_condition = "rain"
            elif "snow" in condition:
                weather_condition = "snow"
            else:
                weather_condition = "sun"

            # Return the weather data as a dictionary
            return {
                'date': date_str,
                'precipitation': precipitation,
                'temp_max_celcius': temp_max,
                'temp_min_celcius': temp_min,
                'wind': wind,
                'weather_condition': weather_condition
            }
        else:
            print(f"Missing 'forecast' data for {date_str}: {data}")
            return None
    else:
        print(f"Failed to retrieve data for {date_str}. Status code: {response.status_code}")
        return None

# Generate the date range for January to September 2024
dates = generate_date_range('2024-01-01', '2024-10-17')

# Collect historical weather data
historical_data = []

for date in dates:
    weather_data = fetch_historical_weather(date)
    if weather_data:
        historical_data.append(weather_data)

# Convert the data into a DataFrame and save to Excel
df = pd.DataFrame(historical_data)
df.to_excel('historical_weather_Vijayawada_data.xlsx', index=False)

print("Historical weather data has been saved to 'historical_weather_Vijayawada_data.xlsx'.")


Historical weather data has been saved to 'historical_weather_Vijayawada_data.xlsx'.
